# HSMA Introduction to Prophet
### Code along lecture 3: Adding 'holidays' to a Prophet model.

AKA Spiky events that we need to model. This is done by adding dummy variables 0 when not that holiday and 1 for holiday.

**In this code along lecture you will learn:**
* How to model special calender events.
---

### Library imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Prophet Imports

In [ ]:
import prophet

from prophet import Prophet
from prophet.plot import (plot_plotly,
                          plot_components_plotly,
                          plot_forecast_component)

# should be version 1.0 or above
prophet.__version__

### Utility functions

In [ ]:
def prophet_training_data(y_train):
    '''
    Converts a standard pandas datetimeindexed dataframe
    for time series into one suitable for Prophet
    Parameters:
    ---------
    y_train: pd.DataFrame
        univariate time series data

    Returns:
    --------
        pd.DataFrame in Prophet format
        columns = ['ds', 'y']
    '''
    prophet_train = pd.DataFrame(y_train.index)
    prophet_train['y'] = y_train.to_numpy()
    prophet_train.columns = ['ds', 'y']

    return prophet_train

### **Step 1**: Load and preprocess data

In [ ]:
url = 'https://raw.githubusercontent.com/hsma4/module_9_b/master/data/' \
      + 'resp_admits_day.csv'
y_train = pd.read_csv(url,
                      parse_dates=True,
                      dayfirst=True,
                      index_col='date')
y_train.index.freq = 'D'

prophet_train = prophet_training_data(y_train)
prophet_train.head()


### **Step 2**: Fit a Prophet model with built-in holidays

As this is ED reattendance data at the daily level, it is likely that we will be seeing some calender day/holiday effects.  Prophet has been designed to deal with 'holidays'.  Effectively a series of binary variables are added for each holiday.

Prophet has a number of standard holidays built in by country.  To add them in for 'UK' you would use the following code:

```python
model = Prophet(interval_width=0.95)
model.add_country_holidays(country_name='UK')
model.fit(y_train)
```

Note that you call the `add_country_holidays` **before** you fit the model.  This is because it is command telling Prophet what you would like to fit.

To see what models have been fitted you can call:

```python
model.train_holiday_names.to_list()
```

When you plot the components of the model a new panel will be added for holidays.  It can sometimes be a bit difficult to which holidays are having which effect so you can plot individual holidays as follows:

```python
from fbprophet.plot import plot_forecast_component

plot_forecast_component(model, prophet_forecast, 'Christmas Day');
```

In [ ]:
# fit a basic prophet model with 0.95 PIs
model = Prophet(interval_width=0.95,
                daily_seasonality=False)
model.add_country_holidays(country_name='UK')
model.fit(prophet_train)

In [ ]:
# have a look at what holidays were included!
model.train_holiday_names.to_list()

### **Step 3**: Make a prediction and analyse components

In [ ]:
# make prediction
future = model.make_future_dataframe(periods=84)
prophet_forecast = model.predict(future)

In [ ]:
#plot the model components - difficult to see which holiday is which!
model.plot_components(prophet_forecast);

In [ ]:
# plot an individual component (as difficult to unpick above)
plot_forecast_component(model, prophet_forecast, 'Good Friday');

### Side bar: using plotly for interactive 

> This works in Juypter Notebooks.  there may be some issues in Jupyter-Lab.  i.e. it doesn't show up.

In [ ]:
plot_plotly(model, prophet_forecast)

### **Step 5:** Plotting Prophet's components

In [ ]:
ax = model.plot_components(prophet_forecast)

### Plotly side bar...

In [ ]:
plot_components_plotly(model, prophet_forecast)

### Adding a manual date

Let's add a random date!

In [ ]:
# black friday dates 2014 - 2025
black_friday = pd.DataFrame({'holiday': 'black_friday',
                             'ds': pd.to_datetime(['2014/11/28', '2015/11/27',
                                                   '2016/11/25', '2017/11/24',
                                                   '2018/11/23', '2019/11/29',
                                                   '2020/11/27', '2021/11/26',
                                                   '2022/11/25', '2023/11/24',
                                                   '2024/11/29', '2025/11/28'])
                                                  })
black_friday

In [ ]:
# pass the manual holidays into a Prophet constructor
model = Prophet(interval_width=0.95,
                daily_seasonality=False,
                holidays=black_friday) # from the DF created above
model.add_country_holidays(country_name='UK')
_ = model.fit(prophet_train)

# make prediction
future = model.make_future_dataframe(periods=84)
prophet_forecast = model.predict(future)

In [ ]:
# check if black friday is now included?
model.train_holiday_names.to_list()

In [ ]:
# conclusion?
# check out the y-axis - Prophet will always try and fit the holidays you add...
# But this may actually reduce accuracy
# Only include holidays that are relevant
plot_forecast_component(model, prophet_forecast, 'black_friday');